In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

# Set figure size to (28,14)
plt.rcParams['figure.figsize'] = (28, 14)

Startin with the EDA

In [ ]:
#read in the csv file as a dataf

In [2]:
df_0 = pd.read_csv("/Users/sebastian/Documents/spiced_projects/euclidean_eukalyptus/work/08_week/weekly_project/Supermarket_Markov/data/monday.csv", sep=';')
df_0.tail()

,timestamp,customer_no,location
4879,2019-09-02 21:49:00,1442,checkout
4880,2019-09-02 21:49:00,1444,checkout
4881,2019-09-02 21:49:00,1445,dairy
4882,2019-09-02 21:50:00,1446,dairy
4883,2019-09-02 21:50:00,1447,fruit


In [3]:
df_0['time'] = pd.to_datetime(df_0['timestamp'])


In [4]:
df_0.set_index('timestamp', inplace=True)

In [5]:
df_0['customer_id']='Mo'+'_'+(df_0['customer_no'].agg(str))

In [11]:
#sort the dataframe by customer_id
def sort_df(df):
    df.sort_index(inplace=True)
    df.sort_values('customer_no', kind='stable', inplace=True)
    return df.head(20)

In [12]:
sort_df(df_0)

,customer_no,location,time,customer_id
timestamp,,,,
2019-09-02 07:03:00,1,dairy,2019-09-02 07:03:00,Mo_1
2019-09-02 07:05:00,1,checkout,2019-09-02 07:05:00,Mo_1
2019-09-02 07:03:00,2,dairy,2019-09-02 07:03:00,Mo_2
2019-09-02 07:06:00,2,checkout,2019-09-02 07:06:00,Mo_2
2019-09-02 07:04:00,3,dairy,2019-09-02 07:04:00,Mo_3
2019-09-02 07:06:00,3,checkout,2019-09-02 07:06:00,Mo_3
2019-09-02 07:04:00,4,dairy,2019-09-02 07:04:00,Mo_4
2019-09-02 07:08:00,4,checkout,2019-09-02 07:08:00,Mo_4
2019-09-02 07:04:00,5,spices,2019-09-02 07:04:00,Mo_5


In [ ]:
df_0.head(20)

In [ ]:
#count the number of unique customers in each ailse of the supermarket

In [9]:
def question_03(df):
    df = pd.DataFrame(df.groupby(['location']).count()['customer_no'])
    return print(df)

In [ ]:
#q4: Calculate the time each customer spent in the market
#groubby location = checkout over time

In [ ]:
def question_04(df):
    #df['minutes_from_timestamps'] = df['timestamp'].dt.minute
    df = pd.DataFrame(df.groupby([pd.Grouper(key='time',freq='min'),'customer_id']))
    return print(df.head(20))

In [7]:
question_03(df_0)

          customer_no
location             
checkout         1437
dairy             895
drinks            797
fruit            1005
spices            750


In [ ]:
df_0.info()

In [15]:
df = df_0.set_index('time')

In [19]:
df.head()

,customer_no,location,customer_id
time,,,
2019-09-02 07:03:00,1,dairy,Mo_1
2019-09-02 07:05:00,1,checkout,Mo_1
2019-09-02 07:03:00,2,dairy,Mo_2
2019-09-02 07:06:00,2,checkout,Mo_2
2019-09-02 07:04:00,3,dairy,Mo_3


In [21]:
#Filling the missing timesteps with pandas resample and ffil()/pad

#df_0.groupby('customer_id')['time'].diff()
df_0 = df.groupby('customer_id').resample('1T').ffill()


In [22]:
df_0

customer_no  location customer_id
customer_id time                                                  
Mo_1        2019-09-02 07:03:00            1     dairy        Mo_1
            2019-09-02 07:04:00            1     dairy        Mo_1
            2019-09-02 07:05:00            1  checkout        Mo_1
Mo_10       2019-09-02 07:06:00           10     fruit       Mo_10
            2019-09-02 07:07:00           10     fruit       Mo_10
...                                      ...       ...         ...
Mo_999      2019-09-02 17:45:00          999     dairy      Mo_999
            2019-09-02 17:46:00          999     fruit      Mo_999
            2019-09-02 17:47:00          999     fruit      Mo_999
            2019-09-02 17:48:00          999     fruit      Mo_999
            2019-09-02 17:49:00          999  checkout      Mo_999

[10765 rows x 3 columns]

Creating the dataframe for the transition matrix

In [ ]:
def markov_columns(df):
    df["before"] = df['location']
    df['after'] = df['location'].shift(-1)
    #df = df.drop(df[(df['before'] == 'checkout')].index)
    #df["after"] = df.apply(lambda x: 'checkout' if x['before'] == 'checkout') 
    #df['after'] = df['before'].apply(lambda x: 'checkout' if (x == 'checkout') else x)
    df.loc[df['before'] == 'checkout', 'after'] = 'checkout'
    #df.set_index('customer_id', inplace=True) # customer_id is needed to arrange the df
    df.drop(columns=['customer_no','location'], inplace=True)
    df.index =pd.to_datetime(df['time'])
    df.drop(columns=['time'], inplace=True)
    #df.sort_index(ascending=False)
    return df.head(10)

In [ ]:
df_time = markov_columns(df_0)


In [ ]:
df_time.head(20)

In [ ]:
df

In [ ]:
#reset of index to datetime is now included in the function

#df_time.set_Date
#df_time['date'] = pd.to_datetime(df_time['time'])
#df_time.index =pd.to_datetime(df_time['date'])
#df_time.drop['']

In [ ]:
#df_time.resample('min', on='time', freq='0:01')